In [17]:
import os
import json
from langchain_openai import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.docstore.document import Document
from langchain.retrievers import EnsembleRetriever
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from common import AgentStreamParser

from kiwipiepy import Kiwi
from common import load_prompt
from dotenv import load_dotenv

load_dotenv()
kiwi = Kiwi()

In [18]:
search = TavilySearchResults(k=3)

In [19]:
DATA_DIR = "./data"

count = 0
for dir in os.listdir(DATA_DIR):
    file_dir = os.path.join(DATA_DIR, dir)
    if dir in [".DS_Store"]:
        continue
    for filename in os.listdir(file_dir):
        count += 1

print(count)

293


In [20]:
DATA_DIR = "./data"

documents = []
for dir in os.listdir(DATA_DIR):
    file_dir = os.path.join(DATA_DIR, dir)
    # if dir in [".DS_Store", "1981~2016", "2017", "2018"]:
    if dir in [".DS_Store"]:
        continue
    for filename in os.listdir(file_dir):
        file_path = os.path.join(file_dir, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            text = json.dumps(data, ensure_ascii=False)
            documents.append(
                {
                    "content": text,
                    "metadata": {
                        "file_path": file_path,
                        "info": {
                            "caseField": data.get("info", {}).get("caseFleid", ""),
                            "detailField": data.get("info", {}).get("detailField", ""),
                            "trailField": data.get("info", {}).get("trailField", ""),
                            "caseNm": data.get("info", {}).get("caseNm", ""),
                            "courtNm": data.get("info", {}).get("courtNm", ""),
                            "judmnAdjuDe": data.get("info", {}).get("judmnAdjuDe", ""),
                            "caseNo": data.get("info", {}).get("caseNo", ""),
                            "relateLaword": data.get("info", {}).get(
                                "relateLaword", ""
                            ),
                        },
                    },
                }
            )

In [21]:
def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]

In [22]:
docs = [
    Document(page_content=doc["content"], metadata=doc["metadata"]) for doc in documents
]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=10)
split_documents = text_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings()

kiwi_bm25 = BM25Retriever.from_documents(
    split_documents, preprocess_func=kiwi_tokenize, k=3
)
faiss = FAISS.from_documents(
    documents=split_documents, embedding=embeddings
).as_retriever(search_kwargs={"k": 3, "fetch_k": 10})

kiwibm25_faiss_73 = EnsembleRetriever(
    retrievers=[kiwi_bm25, faiss], weights=[0.7, 0.3], search_type="mmr"
)

retriever = kiwibm25_faiss_73

In [23]:
retriever_tool = create_retriever_tool(
    retriever,
    name="legal_case_search",
    description="use this tool to search information from the legal document",
)

In [24]:
tools = [search, retriever_tool]

In [25]:
prompt = load_prompt("./prompts/prompt_agent.yaml", encoding="utf-8")

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

agent = create_tool_calling_agent(llm, tools, prompt)

In [26]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    max_iterations=5,
    early_stopping_method="force",
    verbose=False,
)

In [27]:
agent_stream_parser = AgentStreamParser()

In [28]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {
        "question": "피해자의 집에서, 피해자의 몸 위에 올라타고 피해자의 바지와 속옷을 벗기고 피고인을 밀쳐내려고 하는 피해자의 양 손목을 잡아 위로 올려 누르고 몸으로 피해자의 몸을 눌러 저항하지 못하게 한 후 피고인의 성기를 피해자의 음부에 삽입한 사건이 발생했어. 이때 관련된 법이 어떤게 있는지 알려줘. 그리고 유사한 사례가 있는지도 찾아줘"
    }
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: legal_case_search
query: 피해자의 집에서 성폭행 사건 관련 법
Log: 
Invoking: `legal_case_search` with `{'query': '피해자의 집에서 성폭행 사건 관련 법'}`



[도구 호출]
Tool: tavily_search_results_json
query: 유사한 성폭행 사건 사례
Log: 
Invoking: `tavily_search_results_json` with `{'query': '유사한 성폭행 사건 사례'}`



[관찰 내용]
Observation: H과는 관계가 없다고 진술하다.가, 당심에 이르러서는 이를 번복하여 D의 지시에 의하여 그렇게 한 것이라고 진술하고 있는데, 이전까지의 일관된 취지의 진술 내용에 비추어 번복된 위 진술은 쉽사리 믿기 어렵다.", "피고인은 피해자에게 물을 붓는 등의 가해행위를 한 후 피해자의 상태가 나빠지자, 피해자가 피고인을 성폭행하려다가 피고인이 이를 모면하는 과정에서 정당방위로 뜨거운 물을 뿌리게 되었다는 상황을 작출하기 위하여 2013. 6. 27. 01:07경 및 12:12경 H 등과 함께 2차례에 걸쳐 휴대전화로 피고인이 옷을 벗고 있는 상태에서 피해자가 성폭행을 시인하는 모습을 동영상으로 촬영하였다. ", "또한 피고인은 피해자의 화상이 심하여 H, D가 피해자를 병원에 데려가려 하고, 피해자도 너무 아프다며

"AA의 소유관계 다음 표에서 나타나는 바와 같이 2010년 무렵 AA의 주주 구성을 보면, 주주들 중에서 AN(Z의 차남), AO(Z의 장남), 주식회사 AQ(피해자 교회 관련 지주회사)는 피해자 교회에 실질적 영향력을 행사하던 7과 관련이 있다고 보인다. 그러나 이러한 일부 주주 구성만으로는 피해자 교회가 새를 실질적으로 소유하면서 운영하였다고 단정할 수 없다.", "오히려 피고인 B은 교회 건축추진위원장 이자 AA의 대표이사로서, 피해자 교회와 별도로 설립된 사업시행사인 AA를 통하여 이 사건 교회신축사업을 독자적 

In [29]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {
        "question": "피고인은 폭행으로 피해자에 대하여 성기에 손가락을 넣는 행위를 하였을 때 관련된 법에 대해 알려줘"
    }
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)

[최종 답변]
형법 제297조의2에 따르면, 폭행 또는 협박으로 사람에 대하여 성적 수치심이나 혐오감을 일으키는 행위를 한 경우에는 강제추행죄가 성립할 수 있습니다.


In [30]:
chunks = []

for chunk in agent_executor.stream(
    {
        "question": "피고인은 폭행으로 피해자에 대하여 성기에 손가락을 넣는 행위를 하였을 때 관련된 법에 대해 알려줘"
    }
):
    chunks.append(chunk)
print(chunks)

[{'output': '형법 제297조의2에 따르면, 폭행 또는 협박으로 사람에 대하여 성적 수치심이나 혐오감을 일으키는 행위를 한 경우에는 처벌을 받을 수 있습니다. 이는 성폭력범죄의 처벌 등에 관한 특례법에 의해 더욱 강화된 처벌이 적용될 수 있습니다.', 'messages': [AIMessage(content='형법 제297조의2에 따르면, 폭행 또는 협박으로 사람에 대하여 성적 수치심이나 혐오감을 일으키는 행위를 한 경우에는 처벌을 받을 수 있습니다. 이는 성폭력범죄의 처벌 등에 관한 특례법에 의해 더욱 강화된 처벌이 적용될 수 있습니다.', additional_kwargs={}, response_metadata={})]}]


In [35]:
for chunk in chunks:
    if "output" in chunk:
        print(chunk.get("output"))

형법 제297조의2에 따르면, 폭행 또는 협박으로 사람에 대하여 성적 수치심이나 혐오감을 일으키는 행위를 한 경우에는 처벌을 받을 수 있습니다. 이는 성폭력범죄의 처벌 등에 관한 특례법에 의해 더욱 강화된 처벌이 적용될 수 있습니다.


In [36]:
chunks = []

for chunk in agent_executor.stream(
    {
        "question": "피고인은 폭행으로 피해자에 대하여 성기에 손가락을 넣는 행위를 하였을 때 처벌될 수 있는 형량에 대해 알려줘"
    }
):
    chunks.append(chunk)
for chunk in chunks:
    if "output" in chunk:
        print(chunk.get("output"))

형법 제297조의2에 따르면, 폭행 또는 협박으로 사람의 신체에 성기를 넣거나 성기, 항문에 손가락 등 신체의 일부를 넣는 행위는 2년 이상의 유기징역에 처하도록 규정되어 있습니다. 이는 강제추행 중 강간에 준하는 행위로 평가되어 더 중하게 처벌됩니다.


In [38]:
import nltk

nltk.download("wordnet")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mmqq2\AppData\Roaming\nltk_data...


True

In [39]:
from nltk.corpus import wordnet as wn

wn.ensure_loaded()

In [84]:
json_string = json.dumps(documents[0].get("content"), ensure_ascii=False)
data = json.loads(json.loads(json_string)).get("facts").get("bsisFacts")
print(data)

['피고인은 공소외 00, 00, 00, 00, 00과 공모하여 피고인은 형광칩을 삽입하여 패의 투시가 가능하도록 한 특수화투를 제작하고, 위 00은 카메라, 노트북컴퓨터, 화면카트기 등 화투투시용 기기 20여 종을 준비하고, 위 00, 00는 몰래 카메라를 조작하면서 화투의 패를 읽고, 위 00는 직접 이른바 선수로 참여하여 화투를 하였다.', '위 00, 00은 이른바 선수를 모집하기로 역할을 분담하여, 1. 2000. 3. 10. 15:00경부터 같은 날 21:00경까지 사이에 부산 00구 00동에 있는 00모텔 502호실에서, 위 00, 00, 00는 위 모텔 502호실 및 403호실을 빌려 502호실 에어컨 속에 소형 카메라를 설치하고, 403호실에 이 카메라를 통하여 도박현장을 볼 수 있도록 모니터, 노트북컴퓨터, 화면카트기 등을 설치하여 위 카메라와 연결하여 놓았다.', '위 00, 00은 사기도박을 할 상대로 피해자 00, 00, 00를 모집하여 온 다음, 위 00은 사기도박을 할 상대로 피해자 00, 00, 00를 모집하여 온 다음, 위 00가 사실은 위와 같이 파개 투시되는 특수화투를 이용하여 승패를 조작할 것임에도 불구하고 마치 우연하게 승부가 결정되는 것인 양 행세하였다.', '위 피해자들과 속칭 도리짓고땡이라는 도박을 하면서, 위 00, 00이 위 카메라로 바닥에 깔린 화투를 투시하여 패를 읽은 뒤 00가 소지하고 잇는 신호기로 신호를 보내고, 00는 그 신호에 따라 패를 확인하는 수법으로 사기도박을 하여 피해자 00로부터 500만 원을, 같은 00으로부터 500만 원을, 같은 00로부터 1,000만 원을, 각 따서 이를 편취하였다.', '2000. 3. 14. 15:00경부터 같은 날 22:00경까지 사이에 같은 장소에서, 피해자 00, 00, 00를 상대로 제1항 기재와 같은 방법으로 사기도박을 하여 피해자 00로부터 600만 원을, 같은 00로부터 300만 원을, 각 따서 이를 편취하였다.', '2000. 3. 15. 14:00경부

In [91]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


def make_question_data(documents):
    dataset_prompt = PromptTemplate.from_template(
        """
    Please summarize the sentence so that I can use it in a query that asks how many years I will be sentenced to legally.
    Please translate in Korean.

    Question: {question}
    """
    )

    llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

    dataset_chain = dataset_prompt | llm | StrOutputParser()

    json_string = json.dumps(documents.get("content"), ensure_ascii=False)
    data = json.loads(json.loads(json_string)).get("facts").get("bsisFacts")
    question_data = dataset_chain.invoke({"question": str(data)})

    return question_data

In [92]:
print(make_question_data(documents[0]))

질문: 피고인은 공범들과 함께 특수화투와 도박 장비를 사용하여 사기도박을 벌였고, 여러 피해자들로부터 총 3,400만 원을 편취하였으며, 마지막 시도는 단속으로 인해 미수에 그쳤습니다. 이로 인해 피고인은 법적으로 몇 년의 형을 선고받을 수 있습니까?


In [ ]:
temp_data = make_question_data(documents[0])
print(temp_data)
result = agent_executor.stream({"question": temp_data})


for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)

질문: 피고인은 공범들과 함께 특수화투와 도박 장비를 사용하여 사기도박을 벌였고, 여러 피해자들로부터 총 3,400만 원을 편취하였으며, 마지막 시도는 미수에 그쳤습니다. 이로 인해 법적으로 몇 년의 형을 선고받을 수 있습니까?
[도구 호출]
Tool: legal_case_search
query: 사기도박 형량
Log: 
Invoking: `legal_case_search` with `{'query': '사기도박 형량'}`



[관찰 내용]
Observation: 같은 날 22:00경까지 사이에 같은 장소에서, 피해자 00, 00, 00를 상대로 제1항 기재와 같은 방법으로 사기도박을 하여 피해자 00로부터 600만 원을, 같은 00로부터 300만 원을, 각 따서 이를 편취하였다.", "2000. 3. 15. 14:00경부터 같은 날 21:00경까지 사이에 같은 장소에서 피해자 00, 00을 상대로 제1항 기재와 같은 방법으로 사기도박을 하여 피해자 00로부터 500만 원을, 같은 00으로부터 500만 원을 각 따서 이를 편취하였다.", "2000. 3. 17. 15:20경 같은 장소에서, 피해자 00, 00을 상대로 제1항 기재와 같은 방법으로 사기도박을 하여 위 피해자들로부터 돈을 편취하려고 하였으나 단속으로 인해 검거되는 바람에 그 뜻을 이루지 못하고 미수에

할 상대로 피해자 00, 00, 00를 모집하여 온 다음, 위 00은 사기도박을 할 상대로 피해자 00, 00, 00를 모집하여 온 다음, 위 00가 사실은 위와 같이 파개 투시되는 특수화투를 이용하여 승패를 조작할 것임에도 불구하고 마치 우연하게 승부가 결정되는 것인 양 행세하였다.", "위 피해자들과 속칭 도리짓고땡이라는 도박을 하면서, 위 00, 00이 위 카메라로 바닥에 깔린 화투를 투시하여 패를 읽은 뒤 00가 소지하고 잇는 신호기로 신호를 보내고, 00는 그 신호에 따라 패를 확인하는 수법으로 사기도박을 하여 피해자 00로부터 500만 원을, 같은 00으로부터 500만 원을, 같은 00

Stopping agent prematurely due to triggering stop condition


[관찰 내용]
Observation: 같은 날 22:00경까지 사이에 같은 장소에서, 피해자 00, 00, 00를 상대로 제1항 기재와 같은 방법으로 사기도박을 하여 피해자 00로부터 600만 원을, 같은 00로부터 300만 원을, 각 따서 이를 편취하였다.", "2000. 3. 15. 14:00경부터 같은 날 21:00경까지 사이에 같은 장소에서 피해자 00, 00을 상대로 제1항 기재와 같은 방법으로 사기도박을 하여 피해자 00로부터 500만 원을, 같은 00으로부터 500만 원을 각 따서 이를 편취하였다.", "2000. 3. 17. 15:20경 같은 장소에서, 피해자 00, 00을 상대로 제1항 기재와 같은 방법으로 사기도박을 하여 위 피해자들로부터 돈을 편취하려고 하였으나 단속으로 인해 검거되는 바람에 그 뜻을 이루지 못하고 미수에

할 상대로 피해자 00, 00, 00를 모집하여 온 다음, 위 00은 사기도박을 할 상대로 피해자 00, 00, 00를 모집하여 온 다음, 위 00가 사실은 위와 같이 파개 투시되는 특수화투를 이용하여 승패를 조작할 것임에도 불구하고 마치 우연하게 승부가 결정되는 것인 양 행세하였다.", "위 피해자들과 속칭 도리짓고땡이라는 도박을 하면서, 위 00, 00이 위 카메라로 바닥에 깔린 화투를 투시하여 패를 읽은 뒤 00가 소지하고 잇는 신호기로 신호를 보내고, 00는 그 신호에 따라 패를 확인하는 수법으로 사기도박을 하여 피해자 00로부터 500만 원을, 같은 00으로부터 500만 원을, 같은 00로부터 1,000만 원을, 각 따서 이를 편취하였다.", "2000. 3. 14. 15:00경부터 같은 날

당연히 구현되어야 하지만, 국민의 사생활 영역에 관계된 모든 증거의 제출이 곧바로 금지되는 것으로 볼 수는 없으므로 법원으로서는 효과적인 형사소추 및 형사소송에서 진실발견이라는 공익과 개인의 인격적 이익 등 보호이익을 비교형량하여 그 허용 여부를 결정하여야 한다. ", "이때 법원이 그 비교형량을 함에 있어서는 증거수집 절

In [93]:
from nltk.translate import meteor_score

question_data = make_question_data(documents[0])
chunks = []

for chunk in agent_executor.stream({"question": question_data}):
    chunks.append(chunk)
for chunk in chunks:
    if "output" in chunk:
        pred = chunk.get("output")
answer = documents[0].get("metadata").get("info").get("relateLaword")

print(pred)
print(answer)

Stopping agent prematurely due to triggering stop condition


Agent stopped due to max iterations.
['형법 제347조', '형법 제352조', '형법 제30조 ', '형법 제37조', '형법 제38조', '형법 제50조', '형법 제62조', '형법 제48조']
